In [181]:
# load data from csv file and save data into separate lists
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.metrics.cluster import normalized_mutual_info_score
from scipy.fftpack import fft, ifft
from sklearn.decomposition import PCA
import warnings
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import DBSCAN
import pickle

from datetime import timedelta
from scipy.fftpack import fft, ifft,rfft
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, RepeatedKFold
from joblib import dump, load
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


In [182]:
insulin_data_df=pd.read_csv('InsulinData.csv', low_memory=False, usecols=['Date','Time','BWZ Carb Input (grams)'])
cgm_data_df=pd.read_csv('CGMData.csv',low_memory=False,usecols=['Date','Time','Sensor Glucose (mg/dL)'])
insulin_data_df['date_time_stamp']=pd.to_datetime(insulin_data_df['Date'] + ' ' + insulin_data_df['Time'])
cgm_data_df['date_time_stamp']=pd.to_datetime(cgm_data_df['Date'] + ' ' + cgm_data_df['Time'])


def mealdata_extraction(insulin_data_df, cgm_data_df, date_id):
    insulin_df=insulin_data_df.copy()
    insulin_df=insulin_df.set_index('date_time_stamp')
    
    # sort by date_time_stamp, 
    find_timestamp_with_2_5_hours_df=insulin_df.sort_values(by='date_time_stamp',ascending=True).dropna()
    find_timestamp_with_2_5_hours_df['BWZ Carb Input (grams)'].replace(0.0,np.nan,inplace=True)
    find_timestamp_with_2_5_hours_df=find_timestamp_with_2_5_hours_df.dropna().reset_index()
    find_timestamp_with_2_5_hours_df=find_timestamp_with_2_5_hours_df.reset_index().drop(columns='index')
    
    valid_timestamp_list=[]
    value=0
    for idx,i in enumerate(find_timestamp_with_2_5_hours_df['date_time_stamp']):
        try:
            value=(find_timestamp_with_2_5_hours_df['date_time_stamp'][idx+1]-i).seconds / 60.0
            if value >= 120:
                valid_timestamp_list.append(i)
        except KeyError:
            break
        # if idx == 2:
        #     break
    
    
    
    list1 = []
    list2 = []
    
    if date_id==1:
        
        for idx,i in enumerate(valid_timestamp_list):
            start=pd.to_datetime(i - timedelta(minutes=30))
            end=pd.to_datetime(i + timedelta(minutes=120))
            time = pd.to_datetime(i)
            get_date=i.date().strftime('%#m/%#d/%Y')
            x=(insulin_data_df.loc[insulin_data_df['Time']==time.strftime('%H:%M:%S')]['BWZ Carb Input (grams)'].dropna().values.tolist())
            if(len(x)==0):
                continue
            list2.append(x)
            list1.append(cgm_data_df.loc[cgm_data_df['Date']==get_date].set_index('date_time_stamp').between_time(start_time=start.strftime('%H:%M:%S'),end_time=end.strftime('%H:%M:%S'))['Sensor Glucose (mg/dL)'].values.tolist())
    else:
        for idx,i in enumerate(valid_timestamp_list):
            start=pd.to_datetime(i - timedelta(minutes=30))
            end=pd.to_datetime(i + timedelta(minutes=120))
            get_date=i.date().strftime('%Y-%m-%d')
            x=(insulin_data_df.loc[insulin_data_df['Time']==time.strftime('%H:%M:%S')]['BWZ Carb Input (grams)'].dropna().values.tolist())
            print(x)
            if(len(x)==0):
                continue
            list2.append(x)
            list1.append(cgm_data_df.loc[cgm_data_df['Date']==get_date].set_index('date_time_stamp').between_time(start_time=start.strftime('%H:%M:%S'),end_time=end.strftime('%H:%M:%S'))['Sensor Glucose (mg/dL)'].values.tolist())
    
    return pd.DataFrame(list1) , pd.DataFrame(list2)

In [183]:
meal_data,amount_meal_data=mealdata_extraction(insulin_data_df,cgm_data_df,1)
amount_meal_data=amount_meal_data.drop([1],axis=1)
meal_data=meal_data.iloc[:,0:30]
# print(amount_meal_data)
# print(meal_data)
meal_data = meal_data.values.tolist()
amount_meal_data = amount_meal_data.values.tolist()

In [184]:
# this func is used to remove the data which contains 'NaN' and only use the first 30 data
def smooth_data(y,x):
    idx = []
    size_y = len(y)
    for i in range (size_y):
        y[i] = y[i][:30]
        y[i] = y[i][::-1]
        if (len(y[i])!= 30):
            idx.append(i)
        elif 'nan' in y[i]:
            idx.append(i)      
    for j in range (len(idx),0,-1):
        del y[idx[j-1]]
        del x[idx[j-1]]
    return y, x

In [185]:
def calc_vel(y):
    l=[]
    for i in range(len(y)-1):
        l.append(y[i+1]-y[i])
    return l

def calc_acc(y):
    l=[]
    for i in range(len(y)-1):
        l.append(y[i+1]-y[i])
    return l
        

def max_vel(y):
    return max(y)

def min_vel(y):
    return min(y)

def avg_vel(y):
    return sum(y)/len(y)

def max_acc(y):
    return max(y)

def min_acc(y):
    return min(y)

def avg_acc(y):
    return sum(y)/len(y)  

In [186]:
x1,x2 = meal_data , amount_meal_data
for i in range(len(x1)):
    for j in range(len(x1[0])):
        x1[i][j] = str(x1[i][j])


x1, x2 = smooth_data(x1, x2)

print("Number of rows from the processed meal data: ",len(x1) )
print("Number of rows from the processed meal amount data: ",len(x2))

Number of rows from the processed meal data:  299
Number of rows from the processed meal amount data:  299


In [187]:
def extract_ground_truth(x2):
    bin_truth = []
    # bin_temp = {}
    for i in range (len(x2)):
        if (int(x2[i][0])>=0) and (int(x2[i][0])<=20):
            bin_truth.append(1)
        elif (int(x2[i][0])>20) and (int(x2[i][0])<=40):
            bin_truth.append(2)
        elif (int(x2[i][0])>40) and (int(x2[i][0])<=60):
            bin_truth.append(3)
        elif (int(x2[i][0])>60) and (int(x2[i][0])<=80):
            bin_truth.append(4)
        elif (int(x2[i][0])>80) and (int(x2[i][0])<=100):
            bin_truth.append(5)
        elif (int(x2[i][0])>100):
            bin_truth.append(6)
    return bin_truth

In [193]:
# Calculating feature matrix

from scipy.stats import entropy
from scipy.stats import iqr
from scipy import signal
import math
final_matrix =[]
for i in range(len(x1)):
    
    y = list(np.asarray(x1[i], dtype=np.float32))
    yy = calc_vel(y)
    max_v = max_vel(yy)
    min_v = min_vel(yy)
    avg_v = avg_vel(yy)
    # print(avg_v)
    yyy = calc_acc(yy)
    max_a = max_acc(yyy)
    min_a = min_acc(yyy)
    avg_a = avg_acc(yyy)
    ent = entropy(y,base=2)
    iq = iqr(y)
    f,Pxx_den = signal.periodogram(y)
    psd1 = sum(Pxx_den[0:5])/5
    psd2 = sum(Pxx_den[5:10])/5
    psd3 = sum(Pxx_den[10:16])/6
    fft = list(np.fft.fft(y))
    fft_6 = []
    while len(fft_6)<6:
        temp = max(fft)
        mag = math.sqrt(np.real(temp)**2+np.imag(temp)**2)
        fft_6.append(mag)
        del fft[fft.index(temp)]
    f1 = [max_v,min_v,avg_v,max_a,min_a,avg_a,ent,iq,fft_6[0],fft_6[1],fft_6[2],fft_6[3],fft_6[4],fft_6[5],psd1,psd2,psd3]
    # print(len(f1))
    f1 = np.asarray(f1, dtype=object)
    # print(len(f1))
    if i == 0:
        final_matrix = f1
    else:
        final_matrix = np.vstack((final_matrix,f1))
for i in range(len(final_matrix[0])):
    final_matrix = normalize(final_matrix, axis=0, norm='max')
print(pd.DataFrame(final_matrix))
# extract feature and save it into feature metricx
with open('final_matrix.pkl','wb') as f:
    pickle.dump(final_matrix, f)
np.savetxt('test.csv', pd.DataFrame(final_matrix[:60]), fmt="%f", delimiter=",")

           0         1         2         3         4         5         6   \
0    0.058296 -0.110169 -0.082540  0.056769 -0.050691  0.162791  0.998796   
1    0.089686 -0.194915  0.130159  0.087336 -0.055300  0.093023  0.984455   
2    0.067265 -0.067797  0.507937  0.039301 -0.064516  0.046512  0.970723   
3    0.017937 -0.101695 -0.374603  0.043668 -0.055300 -0.069767  0.992977   
4    0.143498 -0.127119  0.209524  0.052402 -0.064516 -0.813953  0.995951   
..        ...       ...       ...       ...       ...       ...       ...   
294  0.139013 -0.110169  0.295238  0.104803 -0.156682  0.209302  0.994548   
295  0.116592 -0.186441  0.142857  0.061135 -0.087558 -0.139535  0.986990   
296  0.147982 -0.101695  0.460317  0.117904 -0.142857  0.093023  0.985629   
297  0.089686 -0.110169 -0.003175  0.061135 -0.152074 -0.395349  0.999509   
298  0.121076 -0.144068  0.225397  0.135371 -0.069124 -0.418605  0.994482   

           7         8         9         10        11        12        13  

In [197]:
bin_truth = extract_ground_truth(x2)
print(bin_truth)
print("number of points in Bin 1 ",bin_truth.count(1))
print("number of points in Bin 2 ",bin_truth.count(2))
print("number of points in Bin 3 ",bin_truth.count(3))
print("number of points in Bin 4 ",bin_truth.count(4))
print("number of points in Bin 5 ",bin_truth.count(5))
print("number of points in Bin 6 ",bin_truth.count(6))

bin_truth = np.asarray(bin_truth)

[4, 3, 3, 4, 3, 3, 6, 2, 1, 5, 1, 2, 6, 5, 3, 2, 3, 2, 3, 2, 2, 5, 3, 5, 1, 1, 2, 4, 1, 3, 2, 3, 3, 4, 1, 2, 3, 2, 1, 2, 4, 2, 1, 2, 3, 5, 2, 2, 1, 3, 3, 5, 3, 6, 3, 2, 5, 3, 1, 2, 4, 6, 2, 2, 2, 2, 5, 4, 2, 2, 2, 3, 1, 4, 3, 2, 4, 2, 3, 1, 5, 1, 3, 5, 3, 3, 3, 5, 3, 1, 3, 3, 3, 3, 2, 2, 5, 3, 2, 5, 3, 3, 1, 2, 2, 2, 3, 1, 3, 2, 1, 2, 2, 3, 4, 3, 1, 5, 3, 2, 4, 3, 4, 4, 3, 4, 4, 2, 1, 2, 5, 1, 1, 2, 3, 3, 2, 1, 2, 2, 3, 1, 1, 3, 2, 4, 1, 3, 2, 1, 1, 3, 1, 1, 3, 3, 2, 5, 1, 3, 1, 1, 1, 2, 1, 3, 2, 1, 4, 1, 2, 2, 4, 2, 4, 2, 4, 1, 1, 4, 2, 1, 1, 5, 2, 4, 3, 4, 2, 3, 1, 2, 1, 2, 2, 3, 3, 1, 4, 2, 2, 3, 2, 3, 2, 1, 4, 5, 2, 2, 1, 5, 1, 3, 4, 1, 3, 3, 4, 2, 1, 1, 1, 3, 2, 5, 1, 1, 1, 1, 2, 1, 2, 2, 4, 1, 2, 2, 2, 5, 1, 1, 2, 1, 3, 3, 2, 2, 1, 3, 2, 1, 1, 4, 3, 1, 3, 2, 3, 2, 1, 4, 2, 2, 3, 1, 5, 2, 3, 3, 2, 1, 1, 1, 2, 4, 1, 1, 2, 2, 3, 2, 5, 3, 2, 3, 1, 3, 3, 4, 2, 1, 2, 1, 5, 2, 1, 2, 4]
number of points in Bin 1  74
number of points in Bin 2  90
number of points in Bin 3  74
number of po

In [211]:
def get_BinsMG(result_labels, true_label, clusterNum):
    binResultMG = []
    bins = []
    for i in range(clusterNum):
        binResultMG.append([])
        bins.append([])
    for i in range(len(result_labels)):
        binResultMG[result_labels[i]-1].append(i)
    # print(binResultMG)
    for i in range(clusterNum):
        for j in binResultMG[i]:
            bins[i].append(true_label[j])
    return bins

def compute_SSEValueMG(bin):
    sse = 0
    if len(bin) != 0:
        avg = sum(bin) / len(bin)
        for i in bin:
            sse += (i - avg) * (i - avg)
    return sse

In [234]:
from sklearn.metrics.cluster import contingency_matrix
from sklearn.preprocessing import StandardScaler
clusterNum=6
# k means clustering on df
kMeansModel = KMeans(n_clusters=clusterNum, random_state=0).fit(final_matrix)
groundTruthBins = bin_truth
# trueLabels = np.asarray(groundTruthBins).flatten()
# for i in range(len(trueLabels)):
#     if math.isnan(trueLabels[i]):
#         trueLabels[i] = 1
# print(kMeansModel.labels_)
# print(groundTruthBins)
bins = get_BinsMG(kMeansModel.labels_, groundTruthBins, clusterNum)
print(bins)
kMeansSSE = 0
for i in range(len(bins)):
    kMeansSSE += (compute_SSEValueMG(bins[i]) * len(bins[i]))
kMeansContingency = contingency_matrix(groundTruthBins, kMeansModel.labels_)
entropy, purity_MG = [], []

for cluster in kMeansContingency:
    cluster = cluster / float(cluster.sum())
    tempEntropyMG = 0
    for x in cluster :
        if x != 0 :
            tempEntropyMG = (cluster * [math.log(x, 2)]).sum()*-1
        else:
            tempEntropyMG = cluster.sum()
    cluster = cluster*3.5
    entropy += [tempEntropyMG]
    purity_MG += [cluster.max()]
counts = np.array([c.sum() for c in kMeansContingency])
coeffs = counts / float(counts.sum())
kMeansEntropy = (coeffs * entropy).sum()
kMeanspurity_MG = (coeffs * purity_MG).sum()
# print(kMeansSSE)
# print(kMeansEntropy)
# print(kMeanspurity_MG)


# dbscan
# X = StandardScaler().fit_transform(final_matrix)
# dbScanModel = DBSCAN(eps=0.05, min_samples=2).fit(final_matrix)
dbScanModel = DBSCAN(eps=0.5, min_samples=2).fit(final_matrix)
print(dbScanModel.labels_)
bins = get_BinsMG(dbScanModel.labels_, groundTruthBins, clusterNum)
dbscanSSE = 0
for i in range(len(bins)):
     dbscanSSE += (compute_SSEValueMG(bins[i]) * len(bins[i]))
dbscanContingency = contingency_matrix(groundTruthBins, dbScanModel.labels_)
entropy, purity_MG = [], []

for cluster in dbscanContingency:
    cluster = cluster / float(cluster.sum())
    tempEntropyMG = 0
    for x in cluster :
        if x != 0 :
            tempEntropyMG = (cluster * [math.log(x, 2)]).sum()*-1
        else:
            tempEntropyMG = (cluster * [math.log(x+1, 2)]).sum()*-1
    entropy += [tempEntropyMG]
    purity_MG += [cluster.max()]
counts = np.array([c.sum() for c in kMeansContingency])
coeffs = counts / float(counts.sum())
dbscanEntropy = (coeffs * entropy).sum()
dbscanpurity_MG = (coeffs * purity_MG).sum()

resultDF=pd.DataFrame([kMeansSSE, dbscanSSE, kMeansEntropy, dbscanEntropy, kMeanspurity_MG, dbscanpurity_MG]).T
resultDF.to_csv('Results_Ravi.csv', header = False, index = False)

[[3, 3, 1, 2, 6, 5, 3, 5, 2, 1, 2, 3, 3, 4, 2, 2, 2, 5, 1, 2, 5, 3, 6, 2, 4, 2, 2, 1, 2, 2, 3, 5, 3, 3, 2, 5, 2, 1, 4, 3, 1, 3, 4, 4, 2, 2, 5, 1, 3, 3, 3, 4, 2, 1, 3, 1, 3, 5, 1, 3, 1, 1, 1, 2, 1, 2, 2, 2, 4, 1, 4, 3, 4, 3, 2, 1, 2, 2, 3, 3, 1, 4, 2, 2, 2, 2, 1, 4, 2, 1, 3, 2, 5, 4, 1, 3, 2, 1, 3, 1, 1, 3, 1, 2, 3, 1, 1, 1, 3, 1, 3, 2, 1, 2], [2, 5, 5, 3, 3, 2, 3, 1, 2, 3], [4, 4, 2, 3, 5, 1, 1, 2, 2, 5, 3, 1, 3, 3, 3, 2, 5, 1, 3, 1, 2, 4, 3, 1, 1, 3, 3, 1, 4, 1, 1, 4, 5, 2, 2, 2, 5, 2, 4, 2, 2, 3, 2, 2, 2], [3, 6, 2, 5, 2, 2, 2, 4, 3, 1, 1, 3, 3, 2, 2, 4, 3, 3, 3, 2, 3, 3, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 3, 1, 3, 4, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 2, 5, 3, 3, 4, 2, 4], [3, 1, 3, 3, 1, 3, 2, 2, 2, 3, 3, 3, 4, 3, 4, 1, 5, 3, 3, 1, 3, 2, 2, 2, 4, 2, 3, 2, 2, 2, 1, 1, 4, 4, 4, 2, 2, 1, 3, 3, 5, 2, 1, 1, 1, 2, 2, 3, 2, 3, 3, 1, 2, 5, 1, 4, 2, 2, 1, 5, 1], [4, 1, 6, 2, 5, 4, 1, 2, 2, 5, 3, 2, 4, 1, 2]]
34722.00000000004
2.262867155907293
1.334448160535117
[ 0  0  0  0  0  0  0  0  0  0  0  0

C:\Users\rguntak1\AppData\Local\jupyterlabdesktopappserver\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(
